# Article classification

Вот и настал момент, который, наверно, ждали многие! Сейчас мы попробуем обучить простую модель для классификации статей. Во многом это туториал вдохновлен <a href="http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html">этими материалами</a>.

### 1. Обучающая выборка

Для начала нам необходимо раздобыть обучающую выборку. Мы воспользуемся готовой коллекцией.

In [1]:
import sklearn.datasets

dataset = sklearn.datasets.fetch_20newsgroups(subset='all', data_home='data')
print('Description:', dataset['description'])

Description: the 20 newsgroups by date dataset


Как видно из описания, данные представляют собой список новостных статей, разбитых на 20 тем:

In [2]:
topics = dataset['target_names']
print('Topics:')
for pair in enumerate(topics, 1):
    print('%i. %s' % pair)

Topics:
1. alt.atheism
2. comp.graphics
3. comp.os.ms-windows.misc
4. comp.sys.ibm.pc.hardware
5. comp.sys.mac.hardware
6. comp.windows.x
7. misc.forsale
8. rec.autos
9. rec.motorcycles
10. rec.sport.baseball
11. rec.sport.hockey
12. sci.crypt
13. sci.electronics
14. sci.med
15. sci.space
16. soc.religion.christian
17. talk.politics.guns
18. talk.politics.mideast
19. talk.politics.misc
20. talk.religion.misc


Но мы хотим научиться решать задачу бинарной классификации. И предсказывать вероятность, что статья нам понравится. Поэтому обсуловимся, что все статьи с темой наука и компьютеры нам интересны, а остальные нет.

In [3]:
interesting, boring = set(), set()
for i, topic in enumerate(topics):
    if topic.startswith('sci.') or topic.startswith('comp.'):
        interesting.add(i)
    else:
        boring.add(i)

Сколько у нас всего статей?

In [4]:
data = dataset['data']
N = len(data)
print(N)

18846


Теперь нам надо позаботиться об извлечении признаков из текста. Посмотрим, как выглядит случайная статья.

In [5]:
import random

In [8]:
article = dataset['data'][random.randint(0, N - 1)]
for line in article.split('\n'):
    print(line)

From: nlu@Xenon.Stanford.EDU (Nelson Lu)
Subject: Re: Why is Barry Bonds not batting 4th?
Organization: Computer Science Department, Stanford University.
Lines: 37

In article <1r93di$car@apple.com> chuq@apple.com (Chuq Von Rospach) writes:
>punjabi@leland.Stanford.EDU (sanjeev punjabi) writes:
>
>>Some evidence that is NOT working:
>
>Take a look at the standings. It's REAL easy to get so focussed on 
>minutinae and forget that the Giants happen to be in first place. If it's
>working, you don't SCREW IT UP by changing things, just because you think it
>ought to be different.

So, that is the reason why the Toronto Blue Jays *should* keep Alfredo
Griffin, just because it "worked"?

A team winning doesn't mean that everything that it's doing is right.
A team not winning doesn't mean that everything that it's doing is wrong, or
otherwise (to borrow the Sharks' situation) you would say that George Kingston
should be fired.

>Some folks like to argue about theoretical details. I prefer to 

Мы видим, что это достаточно грязный текст. Это значит, что для извлечения слова нам потребуются регулярные выражения.

In [12]:
import re

Мы выберем самый простой шаблон. Слово -- это просто непрерывная последовательность букв. Кстати, для простоты сделаем все буквы в тексте маленькими.

In [30]:
WORD_PATTERN = '[a-z]+'

Посмотрим, что получится. 

In [31]:
first_20_words = re.findall(WORD_PATTERN, article.lower())[:20]
for word in first_20_words:
    print(word)

from
nlu
xenon
stanford
edu
nelson
lu
subject
re
why
is
barry
bonds
not
batting
th
organization
computer
science
department


Как мы видим, встречается некоторый мусор, но в целом не так плохо. Теперь нам необходимо получить матрицу с признаками. Для этого мы воспользуемся уже готовым <a href="http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn-feature-extraction-text-countvectorizer">классом</a>

In [34]:
import sklearn.feature_extraction.text

Попробуем сделать запуск.

In [36]:
vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(token_pattern=WORD_PATTERN)
X = vectorizer.fit_transform(dataset['data'])

Результатом работы является матрица <a href="http://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix">типа</a>, с размерностью:

In [48]:
print(X.shape)

(18846, 115065)


Теперь нам необходимо подготовить метки классов.

In [62]:
import numpy

In [72]:
Y = numpy.array([1 if t in interesting else 0 for t in dataset['target']])

### 2. Линейный классификатор

Разделим нашу выборку на две части и сделаем пробное обучение.

In [87]:
border = 15000

X_train, X_test = X[:border], X[border:]
Y_train, Y_test = Y[:border], Y[border:]

Для обучения сначала возьмем
<a href="http://www.machinelearning.ru/wiki/index.php?title=Логистическая_регрессия">логистическую регрессию</a>
со стандартными настройками, которая обучается с помощью
<a href="http://www.machinelearning.ru/wiki/index.php?title=Метод_градиентного_спуска">градиентного спуска</a>.

In [88]:
import sklearn.linear_model

Посмотрим, что у нас получится.

In [89]:
cls = sklearn.linear_model.SGDClassifier(loss='log')
cls.fit(X_train, Y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

В качестве метрики качества будем использовать <a href="http://www.machinelearning.ru/wiki/index.php?title=ROC-кривая">AUC</a>. 

In [90]:
import sklearn.metrics

Посмотрим, какого качества мы достигли, проделав такие простые шаги.

In [95]:
metric = sklearn.metrics.roc_auc_score

Y_pred = cls.predict_proba(X_test)[:, 1]

score = metric(Y_test, Y_pred)
print('Score: ', score)

Score:  0.98850517202


### 3. Кросс-валидация

Итак, мы попробовали обучить классификатор и даже определили качество его работы. Попробуем его улучшить, настроив внешнии парамтеры модели с помощью кроссвалидации.

In [96]:
import sklearn.grid_search

Для этого надо определить кое-что поднастроить.

In [126]:
def scorer(estimator, X, Y):
    return metric(Y, estimator.predict_proba(X)[:, 1])

Создадим сетку поиска параметров.

In [131]:
grid = {
    'penalty': ['elasticnet'],
    'alpha': [0.001, 0.0001, 0.00001, 0.000001, 0.0000001],
    'l1_ratio': [0.0, 0.01, 0.05, 0.10, 0.2, 0.3, 0.4, 0.5],
}

Все подготовим.

In [132]:
searcher = sklearn.grid_search.GridSearchCV(
    estimator = sklearn.linear_model.SGDClassifier(loss='log'),
    param_grid = grid,
    scoring = scorer,
    cv = 5,
    n_jobs=1
)

А теперь надо немного подождать...

In [133]:
searcher.fit(X_train, Y_train);

In [134]:
print(searcher.best_score_)
print(searcher.best_params_)

0.994259001165
{'alpha': 1e-05, 'l1_ratio': 0.0, 'penalty': 'elasticnet'}


А теперь проверим качествао на отложенной выборке!

In [136]:
best_cls = searcher.best_estimator_

print(scorer(best_cls, X_test, Y_test))

0.993980388744


Как мы можем заметить, кроссвалидация дает достаточно точную оценку качества на обучающем множестве :)